# Measuring

In [84]:
import requests
import json

In [7]:
test_files = ['friendly_guitar.wav', 'piano_sonata.mp3', 'Sound_Barrier_Gydra_Remix_trimmed.m4a']

In [96]:
def search(file, engine, k, sample_size, n_mfcc, n_average):
    url = 'http://localhost:5000/search'
    files = { 'audiotrack': open('data/' + file, 'rb') }
    data = { 'top_k': k, 'engine': engine, 'sample_size': sample_size, 'n_mfcc': n_mfcc, 'n_average': n_average }
    response = requests.post(url, data=data, files=files)
    return response.json()

## Chromaprint cross correlation
Params: `span`, `step`

In [92]:
all_spans = [10, 20, 40, 80, 160]
all_steps = [1, 2, 4, 8]
all_sample_sizes = [100, 200, 400]
all_n_mfccs = [3, 6, 10]
all_n_averages = [1, 2, 3]

Run database construction script with all parameters combinations followed by search requests

In [109]:
def run_test(sample_sizes, n_mfccs, n_averages):
    engine = 'mfcc'
    report = []
    
    for sample_size in sample_sizes:  
        for n_mfcc in n_mfccs:
            for n_average in n_averages:
                print("sample_size %i, n_mfcc %i, n_average %i" % (sample_size, n_mfcc, n_average))
                output = !docker-compose exec -T server env PARAM_SAMPLE_SIZE=$sample_size PARAM_N_MFCC=$n_mfcc PARAM_N_AVERAGE=$n_average python ./db.py -e $engine -c
                build_time = float(output[-1][-7:-2])

                avg_time = 0
                results = []
                for file in test_files:
                    json_resp = search(file, engine, 5, sample_size, n_mfcc, n_average)
                    results.append(json_resp['data'])
                    avg_time += json_resp['time']
                avg_time /= len(test_files)

                report.append({
                    'sample_size': sample_size,
                    'n_mfcc': n_mfcc,
                    'n_average': n_average,
                    'results': {
                        'build_time': build_time,
                        'avg_time': avg_time,
                        'data': results
                    }
                })
        
    with open('reports/%s.json' % engine, 'w') as outfile:
        json.dump(report, outfile)
        print("Successfuly saved results to %s.json" % engine)

In [112]:
run_test([200], [6], all_n_averages)

sample_size 200, n_mfcc 6, n_average 1
sample_size 200, n_mfcc 6, n_average 2
sample_size 200, n_mfcc 6, n_average 3
Successfuly saved results to mfcc.json
